### Friday, November 17, 2023

Been doing some model cleanup on the hfpt_Oct28 container, and want to re-run this just to ensure all is still well ...

Yup! This still works! Nice!

So THIS notebook was just copied from TheBlokeCodeLlama.ipynb and I want to use this to download the 13B model and compare it against the 34B model ...

So the 13B model has been downloaded and works just fine! NOTE THIS NOTEBOOK DOES NOT USE LANGCHAIN!

### Thursday, November 16, 2023

https://huggingface.co/TheBloke/CodeLlama-34B-Instruct-GPTQ

Let's see if we can get this to run locally, shall we ... 

I am trying this model because it has the most downloads of all his models, AND of all the huggingface models that are 'code' related.

Wow! This actually works on the 4090! I am stunned! 20524MiB of VRAM!


In [ ]:
# docker cp c9b676310ea0://home/rob/Data2/huggingface/transformers/models--TheBloke--CodeLlama-34B-Instruct-GPTQ /home/rob/Data3/huggingface/transformers
# Successfully copied 18.3GB to /home/rob/Data3/huggingface/transformers

# docker cp c9b676310ea0://home/rob/Data2/huggingface/transformers/models--TheBloke--CodeLlama-13B-Instruct-GPTQ /home/rob/Data3/huggingface/transformers
# Successfully copied 7.26GB to /home/rob/Data3/huggingface/transformers

# (base) rob@KAUWITB:~$ docker cp /home/rob/Data3/huggingface/transformers/models--TheBloke--CodeLlama-34B-Instruct-GPTQ c9b676310ea0://home/rob/Data2/huggingface/transformers
# Successfully copied 18.3GB to c9b676310ea0://home/rob/Data2/huggingface/transformers
# (base) rob@KAUWITB:~$ 


In [1]:
# You need this to just target the 4090, othewise the 34B model does NOT load.
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [8]:
!ls /home/rob/Data2/huggingface/transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


models--TheBloke--CodeLlama-13B-Instruct-GPTQ  version.txt
models--TheBloke--CodeLlama-34B-Instruct-GPTQ


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/CodeLlama-34B-Instruct-GPTQ"

# Let's go with the smaller model, shall we ... 
model_name_or_path = "TheBloke/CodeLlama-13B-Instruct-GPTQ"

# # To use a different branch, change revision
# # For example: revision="main"
# model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
#                                              device_map="auto",
#                                              trust_remote_code=False,
#                                              revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

In [5]:
%%time
# To use a different branch, change revision
# For example: revision="main"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

# 16.7s to load, takes up 20132Mib VRAM

# First time to download ...
# model_name_or_path = "TheBloke/CodeLlama-13B-Instruct-GPTQ"
# CPU times: user 43.8 s, sys: 43.2 s, total: 1min 26s
# Wall time: 1h 42min 31s

You have activated exllama backend. Note that you can get better inference
                    speed using exllamav2 kernel by setting `use_exllama_v2=True`.`disable_exllama` will be deprecated
                    in future version.


CPU times: user 43.8 s, sys: 43.2 s, total: 1min 26s
Wall time: 1h 42min 31s


In [6]:
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# model_name_or_path = "TheBloke/CodeLlama-34B-Instruct-GPTQ"
# # To use a different branch, change revision
# # For example: revision="main"
# model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
#                                              device_map="auto",
#                                              trust_remote_code=False,
#                                              revision="main")

# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

prompt = "Tell me about AI"
prompt_template=f'''[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
{prompt}
[/INST]

'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# # Inference can also be done using transformers' pipeline

# print("*** Pipeline:")
# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens=512,
#     do_sample=True,
#     temperature=0.7,
#     top_p=0.95,
#     top_k=40,
#     repetition_penalty=1.1
# )

# print(pipe(prompt_template)[0]['generated_text'])

# 17.3s 10128MiB VRAM

# 22.5s 20524Mib VRAM




*** Generate:
<s> [INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
Tell me about AI
[/INST]

Here is a possible solution to the coding problem:
```
import re

def tell_me_about_ai(text):
    """
    Return a string describing the topic of the text.

    :param text: The text to analyze.
    :type text: str
    :return: A string describing the topic of the text.
    :rtype: str
    """

    # Remove punctuation and convert to lowercase
    text = re.sub(r"[^\w\s]", "", text.lower())

    # Tokenize the text into individual words
    words = text.split()

    # Create a dictionary to store the word frequencies
    word_freq = {}

    # Iterate over the words and update the word frequencies
    for word in words:
        if word in word_freq:
            word_freq[word] += 1
        else:
            word_freq[word] = 1

    # Find the most common word
    most_common_word = max(

In [7]:
# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

print(pipe(prompt_template)[0]['generated_text'])

# 10.1s 10128MiB VRAM

# 11.9s 20524Mib VRAM

*** Pipeline:
[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
Tell me about AI
[/INST]

Here's a possible solution:
```
def tell_me_about_AI(message):
    if message == "What is AI?":
        return "Artificial intelligence (AI) refers to the ability of machines and computers to perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making."
    elif message == "What are some examples of AI?":
        return "Examples of AI include virtual assistants like Siri or Alexa, self-driving cars, and language translation software."
    else:
        return "Sorry, I don't know what you mean by '{}'. Could you rephrase your question?".format(message)
```
This function takes in a string `message` representing the user's input, and returns a string with the appropriate response. The function uses an `if...elif...else` statement to check th